In [4]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn

ImportError: dlopen(/Users/tjsipin/miniforge3/lib/python3.9/site-packages/scipy/spatial/_qhull.cpython-39-darwin.so, 0x0002): Library not loaded: @rpath/liblapack.3.dylib
  Referenced from: /Users/tjsipin/miniforge3/lib/python3.9/site-packages/scipy/spatial/_qhull.cpython-39-darwin.so
  Reason: tried: '/Users/tjsipin/miniforge3/lib/python3.9/site-packages/scipy/spatial/../../../../liblapack.3.dylib' (no such file), '/Users/tjsipin/miniforge3/lib/python3.9/site-packages/scipy/spatial/../../../../liblapack.3.dylib' (no such file), '/Users/tjsipin/miniforge3/bin/../lib/liblapack.3.dylib' (no such file), '/Users/tjsipin/miniforge3/bin/../lib/liblapack.3.dylib' (no such file), '/usr/local/lib/liblapack.3.dylib' (no such file), '/usr/lib/liblapack.3.dylib' (no such file)